# C2. Barrier Option in Libor Market Model 

# Barrier Caplet Pricing Algorithms
Compare:
1. Analytic closed‑form (\~V)  
2. Algorithm 2.1 (weak 1, kick‑back)  
3. Algorithm 2.2 (weak ½, no VR)  
4. Algorithm 2.2 with variance reduction

### Imports

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
import plotly.graph_objects as go
from tqdm import tqdm

In [6]:
L0, H, K = 0.13, 0.28, 0.01
sigma, T = 0.25, 9.0
N_paths = 3000
h_list  = [0.25, 0.125, 0.0625, 0.03125, 0.015625] #[0.25, 0.125, 0.0625]

# Storing results in this list
results = []

### Analytic barrier‑caplet formula (eqs. 4.3–4.5)

In [7]:
def analytic_vtilde_caplet(L, H, K, sigma, tau):
    v = sigma * np.sqrt(tau)
    def d_plus(x):  return (np.log(x) + 0.5*v**2)/v
    def d_minus(x): return (np.log(x) - 0.5*v**2)/v

    term1 = L*(norm.cdf(d_plus(L/K)) - norm.cdf(d_plus(L/H)))
    term2 = -K*(norm.cdf(d_minus(L/K)) - norm.cdf(d_minus(L/H)))
    term3 = -H*(norm.cdf(d_plus(H*H/(K*L))) - norm.cdf(d_plus(H/L)))
    term4 = K*L/H*(norm.cdf(d_minus(H*H/(K*L))) - norm.cdf(d_minus(H/L)))
    return term1 + term2 + term3 + term4

In [8]:
exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact

0.06571345192162645

# Algorithm 2.1 – weak order 1 with kick‑back reflection

In [9]:
def simulate_algo1(L0, H, K, sigma, T, h, N_paths, seed=42):
    """
    Algorithm 2.1 (weak 1, kick‐back reflection).
    Returns (price, stderr, mean_exit_time).
    """
    rng = np.random.default_rng(seed)
    M = int(np.ceil(T / h))
    lnH = np.log(H)
    sqrt_h = np.sqrt(h)
    half_sigma2_h = 0.5 * sigma**2 * h
    # λ√h from (4.12)
    lambda_sqrt_h = -half_sigma2_h + sigma*sqrt_h

    payoffs = np.zeros(N_paths)
    exit_steps = np.zeros(N_paths, dtype=int)

    for i in range(N_paths):
        lnL = np.log(L0)
        for k in range(M):
            if lnL >= lnH + half_sigma2_h - sigma*sqrt_h:
                # in boundary zone
                delta = lnH - lnL
                p = lambda_sqrt_h / (delta + lambda_sqrt_h)
                if rng.random() < p:
                    # knock‐out at barrier
                    exit_steps[i] = k
                    lnL = lnH
                    break
                else:
                    # kick back in
                    lnL -= lambda_sqrt_h
                    xi = 1 if rng.random() < 0.5 else -1
                    lnL += -half_sigma2_h + sigma*sqrt_h*xi
            else:
                # usual Euler step
                xi = 1 if rng.random() < 0.5 else -1
                lnL += -half_sigma2_h + sigma*sqrt_h*xi
        else:
            # survived to maturity
            exit_steps[i] = M
            payoffs[i] = max(np.exp(lnL) - K, 0.0)

    price = payoffs.mean()
    stderr = payoffs.std(ddof=1) / np.sqrt(N_paths)
    mean_exit_time = exit_steps.mean() * h
    return price, stderr, mean_exit_time

In [10]:
results_algo1 = []

for h in h_list:
    p, se, _   = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Algo': p,
        'CI': 1.96*se
    })

results_algo1

[{'h': 0.25, 'Algo': 0.0671626955325963, 'CI': 0.0020738548076747695},
 {'h': 0.125, 'Algo': 0.06549103088704948, 'CI': 0.00202795788665657},
 {'h': 0.0625, 'Algo': 0.06565471589132234, 'CI': 0.002014484118945565},
 {'h': 0.03125, 'Algo': 0.06610594009418236, 'CI': 0.002030469408846287},
 {'h': 0.015625, 'Algo': 0.0654894031441852, 'CI': 0.0019978620265584074}]

### Algorithm 2.2 – weak order 1/2, absorbing barrier (no VR)

In [ ]:
def simulate_algo2(L0, H, K, sigma, T, h, N_paths, seed=42):
    rng = np.random.default_rng(seed)
    M = int(np.ceil(T/h))
    lnH, sqrt_h = np.log(H), np.sqrt(h)
    half_sig2h = 0.5 * sigma**2 * h
    payoffs = np.zeros(N_paths)
    for i in range(N_paths):
        lnL = np.log(L0)
        for k in range(M):
            if lnL >= lnH + half_sig2h - sigma*sqrt_h:
                break
            xi = 1 if rng.random() < 0.5 else -1
            lnL += -half_sig2h + sigma*sqrt_h*xi
        payoffs[i] = max(np.exp(lnL)-K, 0.0) if k == M-1 else 0.0
    price, stderr = payoffs.mean(), payoffs.std(ddof=1)/np.sqrt(N_paths)
    return price, stderr

In [21]:
results_algo2 = []

for h in h_list:
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Algo': p,
        'CI': 1.96*se
    })

results_algo2

[{'h': 0.25, 'Algo': 0.06276817648644427, 'CI': 0.0020176456765123177},
 {'h': 0.125, 'Algo': 0.06301169186866125, 'CI': 0.0020600861476816066},
 {'h': 0.0625, 'Algo': 0.06208582465469845, 'CI': 0.0019592788978218634},
 {'h': 0.03125, 'Algo': 0.06556391051886841, 'CI': 0.002020493810257483},
 {'h': 0.015625, 'Algo': 0.06377240841252307, 'CI': 0.0019911801195465597}]

In [22]:
# Prepare figure
fig = go.Figure()

# Add trace for Algorithm 1
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Algo'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    name='Algorithm 1'
))

# Add trace for Algorithm 2
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Algo'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    name='Algorithm 2'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo2),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact'
))

fig.show()

We see that Algorithm 2.1 is much more accurate than Algorithm 2.2. We also observe “oscillating” convergence which is typical for binary tree methods.

### Algorithm 2.2 with variance reduction (add Z from (4.6))

### Variance‑reduction term F(s,L) ≈ -σ ∂V/∂L from (4.7)

In [16]:
def F_variance_reduction(L, H, K, sigma, tau, eps=1e-4):
    V_plus  = analytic_vtilde_caplet(L+eps, H, K, sigma, tau)
    V_minus = analytic_vtilde_caplet(L-eps, H, K, sigma, tau)
    dVdL = (V_plus - V_minus) / (2*eps)
    return -sigma * dVdL

In [17]:
def F_variance_reduction(L, H, K, sigma, tau):
    """
    Exact derivative-based variance-reduction term:
      F(s,L) = -sigma * dVtilde/dL
    for the barrier caplet, using closed-form dV/dL.
    """
    # precompute
    v = sigma * np.sqrt(tau)
    inv_v = 1.0 / v
    lnL = np.log(L)
    
    # helper deltas
    a1 = (lnL - np.log(K) + 0.5*v*v) * inv_v        # δ+(L/K)
    a2 = (lnL - np.log(H) + 0.5*v*v) * inv_v        # δ+(L/H)
    b1 = (lnL - np.log(K) - 0.5*v*v) * inv_v        # δ-(L/K)
    b2 = (lnL - np.log(H) - 0.5*v*v) * inv_v        # δ-(L/H)
    c1 = (np.log(H*H/(K*L)) + 0.5*v*v) * inv_v      # δ+(H^2/(K L))
    c2 = (np.log(H/L)    + 0.5*v*v) * inv_v         # δ+(H/ L)
    d1 = (np.log(H*H/(K*L)) - 0.5*v*v) * inv_v      # δ-(H^2/(K L))
    d2 = (np.log(H/L)    - 0.5*v*v) * inv_v         # δ-(H/ L)

    # Gaussian pdf/ccdf
    phi  = norm.pdf
    Phi  = norm.cdf

    # terms
    T1 = (Phi(a1) - Phi(a2)) + inv_v*(phi(a1) - phi(a2))
    T2 = -(K/(L*v))*(phi(b1) - phi(b2))
    T3 =  (H/(L*v))*(phi(c1) - phi(c2))
    T4 = (K/H)*(Phi(d1) - Phi(d2)) - (K/(H*v))*(phi(d1) - phi(d2))

    dVdL = T1 + T2 + T3 + T4
    return -sigma * dVdL

In [18]:
def simulate_algo2_vr(L0, H, K, sigma, T, h, N_paths, seed=42):
    """
    Algorithm 2.2 with variance reduction and a tqdm progress bar.
    Returns (price, stderr).
    """
    rng = np.random.default_rng(seed)
    M = int(np.ceil(T/h))
    lnH, sqrt_h = np.log(H), np.sqrt(h)
    half_sig2h = 0.5 * sigma**2 * h

    payoffs = np.zeros(N_paths)

    # Wrap the outer loop in tqdm
    for i in tqdm(range(N_paths), desc="Simulating paths", unit="path"):
        lnL = np.log(L0)
        Z   = 0.0

        for k in range(M):
            tau = T - k*h
            boundary = lnH + half_sig2h - sigma*sqrt_h
            if lnL >= boundary:
                break

            xi = 1 if rng.random() < 0.5 else -1
            L  = np.exp(lnL)
            Fk = F_variance_reduction(L, H, K, sigma, tau)

            Z   += Fk * sqrt_h * xi
            lnL += -half_sig2h + sigma*sqrt_h * xi

        payoff = max(np.exp(lnL) - K, 0.0) if k == M-1 else 0.0
        payoffs[i] = payoff + Z

    price  = payoffs.mean()
    stderr = payoffs.std(ddof=1) / np.sqrt(N_paths)
    return price, stderr

In [23]:
results_algo2_vr = []
h_list2  = [0.25, 0.125, 0.0625]

for h in h_list2:
    print(h)
    p, se = simulate_algo2_vr(L0, H, K, sigma, T, h, N_paths)
    results_algo2_vr.append({
        'h': h,
        'Algo': p,
        'CI':1.96*se,
    })

results_algo2_vr

0.25


Simulating paths: 100%|██████████| 3000/3000 [00:38<00:00, 77.54path/s]


0.125


Simulating paths: 100%|██████████| 3000/3000 [01:17<00:00, 38.84path/s]


0.0625


Simulating paths: 100%|██████████| 3000/3000 [02:34<00:00, 19.39path/s]


[{'h': 0.25, 'Algo': 0.06616839901949118, 'CI': 0.015550000031309222},
 {'h': 0.125, 'Algo': 0.0660727212430857, 'CI': 0.015419666402308511},
 {'h': 0.0625, 'Algo': 0.0602098829438728, 'CI': 0.01574920530721758}]

### Compare over a few time‑steps and plot

In [ ]:
# Prepare figure
fig = go.Figure()

# Add trace for Algorithm 1
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Algo'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    name='Algorithm 1'
))

# Add trace for Algorithm 2
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Algo'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    name='Algorithm 2'
))

# Add trace for Algorithm 2 VR
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_vr],
    y=[d['Algo'] for d in results_algo2_vr],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_vr], visible=True),
    name='Algorithm 2 VR'
))

# Update layout
fig.update_layout(
    title='Algo Values vs h with CI Error Bars for Multiple Algorithms',
    xaxis_title='h',
    yaxis_title='Algo Value'
)

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_vr],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo2_vr),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact'
))

fig.show()

We see that Algorithm 2.1 is much more accurate than Algorithm 2.2. We also observe “oscillating” convergence
which is typical for binary tree methods.